In [3]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as compare_ssim
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Define the directory to save the resized and compressed images
output_dir = "output_directory"

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to resize an image to 128x128 pixels
def resize_image(img):
    return cv2.resize(img, (128, 128))

# Function to calculate SSIM and MSE
def calculate_metrics(original_img, compressed_img):
    # Calculate SSIM
    ssim = compare_ssim(original_img, compressed_img, multichannel=True, data_range=original_img.max() - original_img.min())

    # Calculate MSE
    mse = np.mean((original_img - compressed_img) ** 2)

    return ssim, mse

# Load FashionMNIST dataset
transform = transforms.Compose([
    # transforms.ToPILImage(),
    transforms.Resize((28, 28)),
    transforms.ToTensor()
])
fashion_mnist_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

# Lists to store SSIM, MSE, and compression ratio values for each image
ssim_values = []
mse_values = []
compression_ratios = []


In [11]:

# Loop through the dataset and compress each image
for i, (image, _) in enumerate(fashion_mnist_dataset):
    # Convert PyTorch tensor to NumPy array
    original_img = np.transpose(image.numpy(), (1, 2, 0)) * 255  # Convert from (C, H, W) to (H, W, C) and unnormalize

    # Resize and compress the original image
    resized_img = resize_image(original_img)
    
    
    encoded_img = cv2.imencode('.jpg', resized_img, [cv2.IMWRITE_JPEG_QUALITY, 90])[1]
    compressed_img = cv2.imdecode(encoded_img, cv2.IMREAD_COLOR)

    
    
    # Convert the resized and compressed image to grayscale for SSIM calculation
    # original_gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
    print(resized_img.shape)
    # compressed_gray = cv2.cvtColor(compressed_img, cv2.COLOR_BGR2GRAY)
    
    # Calculate SSIM and MSE
    ssim, mse = calculate_metrics(resize_image, compressed_img)
    ssim_values.append(ssim)
    mse_values.append(mse)

    # Calculate compression ratio
    original_size = original_img.nbytes
    compressed_size = len(encoded_img)
    compression_ratio = original_size / compressed_size
    compression_ratios.append(compression_ratio)

    # Save the compressed image
    output_path = os.path.join(output_dir, f"image_{i}.jpg")
    cv2.imwrite(output_path, compressed_img)

# Plot SSIM values
plt.figure(figsize=(10, 5))
plt.plot(ssim_values, marker='o', linestyle='-', color='b')
plt.title('SSIM Values after JPEG Compression and Resizing')
plt.xlabel('Image Index')
plt.ylabel('SSIM')
plt.grid(True)
plt.show()

# Plot MSE values
plt.figure(figsize=(10, 5))
plt.plot(mse_values, marker='o', linestyle='-', color='r')
plt.title('MSE Values after JPEG Compression and Resizing')
plt.xlabel('Image Index')
plt.ylabel('MSE')
plt.grid(True)
plt.show()

# Plot compression ratio values
plt.figure(figsize=(10, 5))
plt.plot(compression_ratios, marker='o', linestyle='-', color='g')
plt.title('Compression Ratio Values after JPEG Compression and Resizing')
plt.xlabel('Image Index')
plt.ylabel('Compression Ratio')
plt.grid(True)
plt.show()

(128, 128)


AttributeError: 'function' object has no attribute 'max'